## Case Study - Freddie Mac Bonds

In [1]:
import warnings

import numpy as np
import pandas as pd
from scipy.stats import norm

warnings.filterwarnings("ignore")

In [2]:
# Path to Excel file containing required callable bond and discount curve data
bond_file = '../data/callable_bonds_2025-02-13.xlsx'
discount_file = '../data/discount_curve_2025-02-13.xlsx'

# Read the Excel file for info and quotes of callable bonds into a pandas DataFrame
callable_bonds_info = pd.read_excel(bond_file, sheet_name=0)
callable_bonds_quotes = pd.read_excel(bond_file, sheet_name=1)

# Read the Excel file for discount curve into a pandas DataFrame
discount_curve = pd.read_excel(discount_file, sheet_name=0)

# Display the DataFrame
display(callable_bonds_info.style.set_caption("Callable Bonds Info"))
display(callable_bonds_quotes.style.set_caption("Callable Bonds Quotes"))
display(discount_curve.head().style.set_caption("Discount Curve"))

,info,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
0,CUSIP,3134GW5F9,3134GWGK6,3134HA4V2
1,Issuer,FREDDIE MAC,FREDDIE MAC,FREDDIE MAC
2,Maturity Type,CALLABLE,CALLABLE,CALLABLE
3,Issuer Industry,GOVT AGENCY,GOVT AGENCY,GOVT AGENCY
4,Amount Issued,30000000,25000000,10000000
5,Cpn Rate,0.009700,0.012500,0.044100
6,Cpn Freq,2,2,2
7,Date Quoted,2025-02-13 00:00:00,2025-02-13 00:00:00,2025-02-13 00:00:00
8,Date Issued,2020-10-28 00:00:00,2020-07-29 00:00:00,2025-01-28 00:00:00
9,Date Matures,2028-01-28 00:00:00,2030-01-29 00:00:00,2030-01-28 00:00:00


,quotes,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
0,Date Quoted,2025-02-13 00:00:00,2025-02-13 00:00:00,2025-02-13 00:00:00
1,TTM,2.954141,4.958248,4.955510
2,Clean Price,90.144000,85.109500,99.893000
3,Dirty Price,90.187111,85.161583,100.089000
4,Accrued Interest,0.043111,0.052083,0.196000
5,YTM Call,54.240686,85.395782,4.448321
6,YTM Maturity,4.572927,4.646847,4.433845
7,Duration,2.917203,4.806058,4.496738
8,Modified Duration,2.851993,4.696929,4.399211
9,Convexity,0.095944,0.248027,0.227173


,ttm,maturity date,spot rate,discount
0,0.500000,2025-08-13 00:00:00,0.043743,0.978597
1,1.000000,2026-02-13 00:00:00,0.042890,0.958451
2,1.500000,2026-08-13 00:00:00,0.042238,0.939228
3,2.000000,2027-02-13 00:00:00,0.041843,0.920515
4,2.500000,2027-08-13 00:00:00,0.041632,0.902117


## 1. Pricing the Callable Bond

### 1.1 Bond Pricing

We will price:
1. The callable bond (FHLMC 4.41 01/28/30)
2. Hypothetical non-callable version with original maturity
3. Hypothetical non-callable version with maturity at call date

All pricing uses the provided discount curve.

In [3]:
# Function to interpolate discount factors
def interpolate_discount_factor(ttm_target, discount_curve):
    """
    Interpolate discount factor for a given time to maturity.
    Uses linear interpolation on log of discount factors.
    """
    ttm_curve = discount_curve['ttm'].values
    discount_factors = discount_curve['discount'].values
    
    if ttm_target <= ttm_curve[0]:
        return np.interp(ttm_target, ttm_curve, discount_factors)
    elif ttm_target >= ttm_curve[-1]:
        return np.interp(ttm_target, ttm_curve, discount_factors)
    else:
        # Linear interpolation on log(discount)
        log_discounts = np.log(discount_factors)
        log_df_interpolated = np.interp(ttm_target, ttm_curve, log_discounts)
        return np.exp(log_df_interpolated)


# Generate cash flow schedule for a bond
def generate_cash_flows(date_start, date_end, coupon_rate, coupon_freq, principal=100):
    """
    Generate cash flow dates and amounts for a bond.
    
    Parameters:
    - date_start: valuation date
    - date_end: maturity date
    - coupon_rate: annual coupon rate
    - coupon_freq: coupons per year
    - principal: face value (default 100)
    
    Returns:
    - DataFrame with columns: date, time_to_cf, coupon_payment, principal_payment, total_cf
    """
    # Create coupon payment dates
    cf_dates = []
    current_date = date_end
    
    while current_date > date_start:
        cf_dates.append(current_date)
        current_date = current_date - pd.DateOffset(months=int(12/coupon_freq))
    
    cf_dates.reverse()
    
    # Calculate time to each cash flow (in years)
    times_to_cf = [(d - date_start).days / 365.25 for d in cf_dates]
    
    # Calculate coupon payments
    coupon_payment = principal * coupon_rate / coupon_freq
    coupon_payments = [coupon_payment] * len(cf_dates)
    
    # Principal payment at maturity
    principal_payments = [0] * (len(cf_dates) - 1) + [principal]
    
    # Total cash flows
    total_cfs = [c + p for c, p in zip(coupon_payments, principal_payments)]
    
    df = pd.DataFrame({
        'date': cf_dates,
        'time_to_cf': times_to_cf,
        'coupon_payment': coupon_payments,
        'principal_payment': principal_payments,
        'total_cf': total_cfs
    })
    
    return df


# Price a bond using discount curve
def price_bond(cash_flows_df, discount_curve):
    """
    Price a bond using cash flows and discount curve.
    
    Parameters:
    - cash_flows_df: DataFrame with 'time_to_cf' and 'total_cf' columns
    - discount_curve: DataFrame with 'ttm' and 'discount' columns
    
    Returns:
    - bond_price: present value of all cash flows
    """
    pv = 0
    for _, row in cash_flows_df.iterrows():
        ttm = row['time_to_cf']
        cf = row['total_cf']
        df = interpolate_discount_factor(ttm, discount_curve)
        pv += cf * df
    
    return pv    


# Extract bond information for FHLMC 4.41 01/28/30
bond_idx = 3
bond_name = 'FHLMC 4.41 01/28/30'

# Bond characteristics
coupon_rate = callable_bonds_info.iloc[5, bond_idx]
coupon_freq = callable_bonds_info.iloc[6, bond_idx]
date_quoted = pd.to_datetime(callable_bonds_info.iloc[7, bond_idx])
date_issued = pd.to_datetime(callable_bonds_info.iloc[8, bond_idx])
date_matures = pd.to_datetime(callable_bonds_info.iloc[9, bond_idx])
date_first_call = pd.to_datetime(callable_bonds_info.iloc[10, bond_idx])
date_next_call = pd.to_datetime(callable_bonds_info.iloc[11, bond_idx])
strike = callable_bonds_info.iloc[12, bond_idx]

# Market quotes
ttm = callable_bonds_quotes.iloc[1, bond_idx]
clean_price = callable_bonds_quotes.iloc[2, bond_idx]
dirty_price = callable_bonds_quotes.iloc[3, bond_idx]
accrued_interest = callable_bonds_quotes.iloc[4, bond_idx]
quoted_duration = callable_bonds_quotes.iloc[7, bond_idx]
implied_vol = callable_bonds_quotes.iloc[12, bond_idx]


# Display bond characteristics
bond_info_dict = {
    "Bond": [bond_name],
    "Coupon Rate": [f"{coupon_rate:.4f} ({coupon_rate*100:.2f}%)"],
    "Coupon Frequency": [f"{coupon_freq} (semi-annual)"],
    "Date Quoted": [date_quoted.date()],
    "Date Matures": [date_matures.date()],
    "Date Next Call": [date_next_call.date()],
    "Strike": [strike],
    "TTM (years)": [f"{ttm:.4f}"],
    "Market Dirty Price": [f"{dirty_price:.6f}"]
}

bond_info_df = pd.DataFrame(bond_info_dict)
display(bond_info_df.style.set_caption("FHLMC 4.41 01/28/30 Bond Characteristics"))

,Bond,Coupon Rate,Coupon Frequency,Date Quoted,Date Matures,Date Next Call,Strike,TTM (years),Market Dirty Price
0,FHLMC 4.41 01/28/30,0.0441 (4.41%),2 (semi-annual),2025-02-13,2030-01-28,2028-01-28,100,4.9555,100.089000


In [4]:
# 1. Market price (callable bond)
callable_bond_price = dirty_price

# 2. Hypothetical non-callable bond (full maturity)
cf_hypothetical_full = generate_cash_flows(
    date_quoted, date_matures, coupon_rate, coupon_freq,
)
price_hypothetical_full = price_bond(cf_hypothetical_full, discount_curve)

# 3. Hypothetical non-callable bond (call date maturity)
cf_hypothetical_call = generate_cash_flows(
    date_quoted, date_next_call, coupon_rate, coupon_freq,
)
price_hypothetical_call = price_bond(cf_hypothetical_call, discount_curve)

# Organize results in a DataFrame for cleaner display
pricing_summary = pd.DataFrame(
    {
        "Scenario": [
            "Market Dirty Price (Callable Bond)",
            f"Hypothetical Non-Callable (to Maturity {date_matures.date()})",
            f"Hypothetical Non-Callable (to Call Date {date_next_call.date()})",
        ],
        "Model Price": [
            f"{callable_bond_price:.6f}",
            f"{price_hypothetical_full:.6f}",
            f"{price_hypothetical_call:.6f}",
        ],
    }
)

display(pricing_summary.style.set_caption(f"Bond pricing summary - {bond_name}"))


# Market Implied Call Value = Hypothetical Non-Callable (to Maturity) - Callable Bond Price
market_implied_call_value = price_hypothetical_full - callable_bond_price

call_value_df = pd.DataFrame(
    {
        "": ["Market Implied Call Value"],
        "Value": [f"{market_implied_call_value:.6f}"],
    }
)

display(call_value_df.style.set_caption(f"Market Implied Call Value - {bond_name}"))

,Scenario,Model Price
0,Market Dirty Price (Callable Bond),100.089000
1,Hypothetical Non-Callable (to Maturity 2030-01-28),101.408472
2,Hypothetical Non-Callable (to Call Date 2028-01-28),100.902111


,,Value
0,Market Implied Call Value,1.319472


### 1.2 Forward Price of Hypothetical Bond at Call Date

Calculate the forward price of the hypothetical bond as of the call date.

The forward price is calculated as:
$$F(0,T_1) = \frac{P_{future}(T_1,T_2)}{Z(0,T_1)}$$

where:
- $P_{future}(T_1,T_2)$ = price at time $T_1$ of bond maturing at $T_2$
- $Z(0,T_1)$ = discount factor from now (time 0) to call date $T_1$


### Understanding Forward Price Calculation

The forward price $F(0,T_1)$ is the price at time $T_1$ (call date) of a bond maturing at $T_2$ (final maturity), as viewed from today (time 0).

**Key Concept:**
The forward price represents what the bond will be worth at the call date, given today's discount curve. It's NOT the discounted value back to today—it's the actual price that will prevail at time $T_1$.

**Mathematical Derivation:**

For a bond with cash flows $CF_i$ at times $t_i$ (where $T_1 < t_i \leq T_2$):

$$F(0,T_1) = \sum_{i} CF_i \cdot Z_{T_1}(t_i)$$

where $Z_{T_1}(t_i)$ is the discount factor **from time $T_1$ to time $t_i$**.

Since we only have discount factors from today (time 0), we use the relationship:

$$Z_{T_1}(t_i) = \frac{Z_0(t_i)}{Z_0(T_1)}$$

This comes from the no-arbitrage condition:
$$Z_0(t_i) = Z_0(T_1) \cdot Z_{T_1}(t_i)$$

In [5]:
# Calculate time to call date in years
time_to_call = (date_next_call - date_quoted).days / 365

# Get the discount factor to the call date (from today)
discount_to_call = interpolate_discount_factor(time_to_call, discount_curve)

# Generate cash flows occurring from call date to final maturity
cf_from_call_to_maturity = generate_cash_flows(
    date_next_call, date_matures, coupon_rate, coupon_freq
)

# Calculate time (in years) from call date for each cash flow
cf_from_call_to_maturity["time_from_call"] = [
    (d - date_next_call).days / 365 for d in cf_from_call_to_maturity["date"]
]

# Initialize present value at call date
price_at_call_date = 0.0

# Loop through each projected cash flow and discount back to the call date
for _, row in cf_from_call_to_maturity.iterrows():
    cf = row["total_cf"]
    time_from_call = row["time_from_call"]
    time_from_quote = time_to_call + time_from_call  # years from today to cash flow

    # Discount factor from today to cash flow
    df_from_quote = interpolate_discount_factor(time_from_quote, discount_curve)
    # Discount factor from call date to cash flow (forward discounting)
    df_from_call = df_from_quote / discount_to_call

    price_at_call_date += cf * df_from_call

# Forward price equals projected price at call date (already discounted appropriately)
forward_price = price_at_call_date

# Organize results in a DataFrame for display
forward_summary_df = pd.DataFrame(
    {
        "Parameter": [
            "Time to Call Date (years)",
            "Discount Factor to Call Date",
            "Forward Price F(0, T1)",
        ],
        "Value": [
            f"{time_to_call:.6f}",
            f"{discount_to_call:.6f}",
            f"{forward_price:.6f}",
        ],
    }
)

display(
    forward_summary_df.style.set_caption(
        f"Forward Price Calculation to Call Date - {bond_name}"
    )
)

,Parameter,Value
0,Time to Call Date (years),2.956164
1,Discount Factor to Call Date,0.885651
2,"Forward Price F(0, T1)",100.566162


### 1.3 Implied Volatility Conversion

The quoted implied volatility (23.88%) corresponds to the volatility of the forward rate.

We need to convert this to the implied volatility of the bond's forward price using the duration approximation:

$$\sigma_{price} \approx D \times \sigma_{rate} \times f(T_1)$$

where:
- $D$ = duration of the bond
- $\sigma_{rate}$ = volatility of the forward rate (given)
- $f(T_1)$ = continuously compounded forward rate at time $T_1$

For simplicity, we can approximate $f(T_1)$ with the spot rate at time $T_1$.

In [6]:
# Extract the spot rate at time T1 (call date)
spot_rate_at_call = interpolate_discount_factor(time_to_call, discount_curve)
# Convert to rate: r = -ln(Z(T))/T
forward_rate_approx = -np.log(spot_rate_at_call) / time_to_call

# Convert implied vol from rate to price
sigma_rate = implied_vol / 100  # Convert from percentage
duration = quoted_duration  # Use quoted duration

sigma_price = duration * sigma_rate * forward_rate_approx

# Organize results in a DataFrame for display
implied_vol_conversion_df = pd.DataFrame(
    {
        "Parameter": [
            "Time to Call Date (years)",
            "Discount Factor at Call",
            "Approximate Forward Rate f(T1)",
            "Implied Vol (rate)",
            "Quoted Duration",
            "Implied Vol (price)",
        ],
        "Value": [
            f"{time_to_call:.6f}",
            f"{spot_rate_at_call:.6f}",
            f"{forward_rate_approx:.6f} ({forward_rate_approx*100:.4f}%)",
            f"{implied_vol:.4f}% = {sigma_rate:.6f}",
            f"{duration:.6f}",
            f"{sigma_price:.6f} = {sigma_price*100:.4f}%",
        ],
    }
)

display(implied_vol_conversion_df.style.set_caption("Implied Volatility Conversion"))

,Parameter,Value
0,Time to Call Date (years),2.956164
1,Discount Factor at Call,0.885651
2,Approximate Forward Rate f(T1),0.041078 (4.1078%)
3,Implied Vol (rate),23.8798% = 0.238798
4,Quoted Duration,4.496738
5,Implied Vol (price),0.044110 = 4.4110%


### 1.4 Callable Bond Valuation Using Black's Formula

The callable bond can be valued as:
$$P_{callable} = P_{straight} - C_{call}$$

where:
- $P_{straight}$ = price of non-callable bond
- $C_{call}$ = value of embedded call option

The call option value is calculated using Black's formula:
$$C = Z(0,T_1) \times [F \cdot N(d_1) - K \cdot N(d_2)]$$

where:
$$d_1 = \frac{\ln(F/K) + 0.5\sigma^2 T_1}{\sigma\sqrt{T_1}}$$
$$d_2 = d_1 - \sigma\sqrt{T_1}$$

- $F$ = forward price
- $K$ = strike price
- $\sigma$ = implied volatility of forward price
- $T_1$ = time to call date
- $N(\cdot)$ = cumulative standard normal distribution

In [10]:
# Black's formula for call option
def blacks_call_option(F, K, sigma, T, Z):
    """
    Calculate call option value using Black's formula.

    Parameters:
    - F: forward price
    - K: strike price
    - sigma: volatility of forward price
    - T: time to expiration
    - Z: discount factor to expiration

    Returns:
    - call_value: value of call option
    - d1: Black-Scholes d1 parameter
    - d2: Black-Scholes d2 parameter
    """
    if T <= 0 or sigma <= 0:
        call_val = max(F - K, 0) * Z
        d1 = float('nan')
        d2 = float('nan')
        return call_val, d1, d2

    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_value = Z * (F * norm.cdf(d1) - K * norm.cdf(d2))
    return call_value, d1, d2


# Calculate call option value
F = forward_price
K = strike
sigma = sigma_price
T = time_to_call
Z = discount_to_call

call_value, d1, d2 = blacks_call_option(F, K, sigma, T, Z)

# Option analytics dataframe
black_formula_df = pd.DataFrame(
    {
        "Parameter": [
            "Forward Price (F)",
            "Strike Price (K)",
            "Volatility (σ)",
            "Volatility (%)",
            "Time to Call (T, years)",
            "Discount Factor Z(0,T)",
            "d1",
            "d2",
            "N(d1)",
            "N(d2)",
            "Call Option Value",
        ],
        "Value": [
            f"{F:.6f}",
            f"{K:.6f}",
            f"{sigma:.6f}",
            f"{sigma*100:.4f}%",
            f"{T:.6f}",
            f"{Z:.6f}",
            f"{d1:.6f}",
            f"{d2:.6f}",
            f"{norm.cdf(d1):.6f}",
            f"{norm.cdf(d2):.6f}",
            f"{call_value:.6f}"
        ],
    }
)
display(black_formula_df.style.set_caption("Black's Formula - Embedded Call Option"))

# Calculate callable bond value and comparison
price_callable_model = price_hypothetical_full - call_value

callable_bond_valuation_df = pd.DataFrame(
    {
        "Parameter": [
            "Non-Callable Bond Price",
            "Embedded Call Value",
            "Callable Bond Price (Model)",
            "Market Price",
            "Model - Market",
            "Percentage Difference",
        ],
        "Value": [
            f"{price_hypothetical_full:.6f}",
            f"{call_value:.6f}",
            f"{price_callable_model:.6f}",
            f"{dirty_price:.6f}",
            f"{price_callable_model - dirty_price:.6f}",
            f"{(price_callable_model - dirty_price)/dirty_price * 100:.4f}%"
        ],
    }
)
display(callable_bond_valuation_df.style.set_caption("Callable Bond Valuation (Model vs Market)"))

,Parameter,Value
0,Forward Price (F),100.566162
1,Strike Price (K),100.000000
2,Volatility (σ),0.044110
3,Volatility (%),4.4110%
4,"Time to Call (T, years)",2.956164
5,"Discount Factor Z(0,T)",0.885651
6,d1,0.112361
7,d2,0.036521
8,N(d1),0.544732
9,N(d2),0.514567


,Parameter,Value
0,Non-Callable Bond Price,101.408472
1,Embedded Call Value,2.944709
2,Callable Bond Price (Model),98.463763
3,Market Price,100.089000
4,Model - Market,-1.625237
5,Percentage Difference,-1.6238%


### 1.5 Yield to Maturity (YTM) Calculations

Calculate YTM under two scenarios:
1. Bond is never called (use full maturity)
2. Bond is certainly called (use call date as maturity, receive strike at call)

Compare to market-quoted YTM Call and YTM Maturity.

### 1.6 Duration Calculations

Calculate duration for:
1. Hypothetical non-callable bond
2. Callable bond (using numerical differentiation)

Duration is calculated as:
$$D = -\frac{1}{P} \frac{dP}{dr}$$

For the callable bond, we perturb the spot curve by ±1bp and recalculate the bond value.

### 1.7 Option-Adjusted Spread (OAS)

The OAS is the parallel shift to the spot curve needed to make the model price equal to the market price.

We solve for OAS such that:
$$P_{model}(OAS) = P_{market}$$

where the model recalculates the callable bond value with the shifted curve.

### 1.8 Optional OTM Callables

In this optional section, we examine two additional Freddie Mac callable bonds that are far **out of the money (OTM)**:

1. **FHLMC 0.97 01/28/28** - 0.97% coupon, maturing January 28, 2028
2. **FHLMC 1.25 01/29/30** - 1.25% coupon, maturing January 29, 2030

**Key Characteristics:**

- **Low Coupons:** Both bonds have very low coupon rates (0.97% and 1.25%) compared to current market rates
- **Deep OTM:** With current rates around 4%, these bonds trade at significant discounts to par
- **Near-term Call Options:** The call options expire in approximately 3 months (though the code adjusts this to 6 months to align with coupon payment dates)
- **Unlikely to be Called:** Since the bonds trade well below par (around 90.14 and 85.11 respectively), and the strike is 100, the issuer has no incentive to call these bonds

**Why These Don't Have Interesting Convexity:**

For a callable bond to exhibit **negative convexity**, the embedded call option must be near-the-money (ATM) or in-the-money (ITM). When a bond is deep OTM:

- The call option has virtually no value (delta ≈ 0)
- The bond behaves almost exactly like a straight (non-callable) bond
- Duration and convexity are similar to non-callable bonds

**Comparison to FHLMC 4.41 01/28/30:**

| Feature | FHLMC 0.97 | FHLMC 1.25 | FHLMC 4.41 (Main) |
|---------|------------|------------|-------------------|
| Coupon | 0.97% | 1.25% | 4.41% |
| Clean Price | ~90.14 | ~85.11 | ~99.89 |
| Moneyness | Deep OTM | Deep OTM | Near ATM/ITM |
| Call Value | ~$0 | ~$0 | Significant |
| Convexity | Positive (normal) | Positive (normal) | **Negative** |
| Behaves Like | Straight bond | Straight bond | Callable bond |

**Key Insight:**

The **negative convexity** of callable bonds—where price appreciation is limited as rates fall—only manifests when the call option has significant value. For deep OTM bonds like these, the probability of call is so low that the embedded option can be effectively ignored for pricing and risk management purposes.

### 1.9 ATM with 1-yr Expiry

For this optional analysis, we examine an alternative dataset with a **recently-issued callable bond** that has more favorable characteristics for observing callable bond behavior:

**Implementation:**

To analyze this bond, we will use the same code developed in sections 1.1-1.7, but with a different input file:
```python
# Use alternative dataset dated 2025-02-18
bond_file = '../data/callable_bonds_2025-02-18.xlsx'
discount_file = '../data/discount_curve_2025-02-18.xlsx'
```

All the pricing methodologies, Black's formula implementation, duration calculations, and OAS calibration remain identical. Only the input data changes.

**Note:** The code and results for this implementation can be found in the file ``` 'options_on_rates_and_bonds/freddie_mac_bonds_alternate_file.ipynb' ```